In [1]:
#pip install pyarrow

In [2]:
#pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.6 MB/s eta 0:00:00


In [3]:
import os
import re
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import (
    precision_score,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    classification_report,
)

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score


In [5]:
pd.options.mode.chained_assignment = None


In [6]:
from urllib.request import urlretrieve
def direct_load(fname):

    if not os.path.exists(f'{PATH}/{fname}'):
        urlretrieve(f'{REPO_URL}/{fname}', f'{PATH}/{fname}')
    df=pd.read_parquet(f'{PATH}/{fname}')
    return df

In [8]:
INSTRUMENT='GRAVITY'
RANGE='1m'
REPO_URL='https://huggingface.co/datasets/Paranal/parlogs-observations/resolve/main/data'
PATH='Data' # Convenient name to be Colab compatible




df_meta = direct_load(f'{INSTRUMENT}-{RANGE}-meta.parket')
df_inst = direct_load(f'{INSTRUMENT}-{RANGE}-traces.parket')
df_subs = direct_load(f'{INSTRUMENT}-{RANGE}-traces-SUBSYSTEMS.parket')
df_tele = direct_load(f'{INSTRUMENT}-{RANGE}-traces-TELESCOPES.parket')

In [9]:
print(len(df_meta))
print(len(df_inst))
print(len(df_subs))
print(len(df_tele))

768
1626954
681968
1232784


In [10]:
df_all = pd.concat([df_inst, df_subs, df_tele]).sort_values('@timestamp').reset_index(drop=True)

# Create event column
df_all["event"] = ""
for col in ['logtype', 'system', 'keywname', 'keywvalue', 'logtext']:
    df_all["event"] += " " + df_all[col]


df_all['tokenized'] = df_all['event'].apply(lambda x: re.sub(r'[\d:-]+', '', x))
# Restrict columns
df_event = df_all[['@timestamp', 'event', 'tokenized']]

with pd.option_context('display.max_colwidth', None):
    display(df_event[-30:])

,@timestamp,event,tokenized
3541676,2019-04-30 22:36:37.650,"ERR GRAVITY caiERR_WRITE : database direct write attribute failed (<alias>P20.totFlagTelSCPer(0:4,T1:T4))","ERR GRAVITY caiERR_WRITE database direct write attribute failed (<alias>P.totFlagTelSCPer(,TT))"
3541677,2019-04-30 22:36:37.650,ERR GRAVITY caiERR_WRITE : database direct write attribute failed (Direct Address: Record=[0;4] - Field=[0;3]),ERR GRAVITY caiERR_WRITE database direct write attribute failed (Direct Address Record=[;] Field=[;])
3541678,2019-04-30 22:36:37.650,ERR GRAVITY caiERR_WRITE : database direct write attribute failed (Direct Address: PLIN=124 - AIN=67),ERR GRAVITY caiERR_WRITE database direct write attribute failed (Direct Address PLIN= AIN=)
3541679,2019-04-30 22:36:37.650,ERR GRAVITY caiERR_WRITE : database direct write attribute failed (Buffer Size=80 bytes - DataType(1st element)=48),ERR GRAVITY caiERR_WRITE database direct write attribute failed (Buffer Size= bytes DataType(st element)=)
3541680,2019-04-30 22:36:37.650,"LOG GRAVITY (errPutInStack) Error Stack -233621517 is FULL, module cai, location caiGetAttributes.c:440","LOG GRAVITY (errPutInStack) Error Stack is FULL, module cai, location caiGetAttributes.c"
3541681,2019-04-30 22:36:37.748,ERR DL dlpxposERR_TEST : Metrology test failed - Low quality,ERR DL dlpxposERR_TEST Metrology test failed Low quality
3541682,2019-04-30 22:36:37.748,ERR DL dlpxposERR_LASER_SIGNAL : Lost Laser Metrology input signal - Check cables,ERR DL dlpxposERR_LASER_SIGNAL Lost Laser Metrology input signal Check cables
3541683,2019-04-30 22:36:37.748,ERR DL dlpxposERR_LASER_GLITCH : Laser Metrology detected 2000 glitches,ERR DL dlpxposERR_LASER_GLITCH Laser Metrology detected glitches
3541684,2019-04-30 22:36:37.748,LOG DL DLXPOS Number of Glitches 2000!,LOG DL DLXPOS Number of Glitches !
3541685,2019-04-30 22:36:37.748,LOG DL DLXPOS Metrology Error!,LOG DL DLXPOS Metrology Error!


In [11]:
import pandas as pd

# Crear la columna TIMEOUT_ERROR en df_meta, donde se verifica si ERROR o TIME_OUT es True
df_meta['TIMEOUT_ERROR'] = (df_meta['ERROR'] | df_meta['TIMEOUT']).astype(int)

In [12]:


# Ahora, une los DataFrames df_all y df_meta usando trace_id como referencia.
# Asumimos que df_meta tiene un índice que se corresponde de alguna manera con df_all["trace_id"].
# Si trace_id está en df_meta como una columna:
df_combined = df_all.merge(df_meta[['TIMEOUT_ERROR']], left_on='trace_id', right_index=True, how='left')

# Si trace_id es el índice en df_meta, podrías usar right_index=True directamente.


In [13]:
df_combined

,@timestamp,system,hostname,loghost,logtype,envname,procname,procid,module,keywname,keywvalue,keywmask,logtext,trace_id,event,tokenized,TIMEOUT_ERROR
0,2019-04-02 02:26:35.908,GRAVITY,wgv,wgv,LOG,wgv,bob_26244,87.0,seq,,,,GRAVITY_single_acq -- GRAVITY Single Mode Acqu...,0,LOG GRAVITY GRAVITY_single_acq -- GRAVITY S...,LOG GRAVITY GRAVITY_single_acq GRAVITY Sin...,1
1,2019-04-02 02:26:35.908,GRAVITY,wgv,wgv,LOG,wgv,bob_26244,87.0,seq,,,,Started at 2019-04-02T02:26:35 (underlined),0,LOG GRAVITY Started at 2019-04-02T02:26:35 ...,LOG GRAVITY Started at T (underlined),1
2,2019-04-02 02:26:36.000,GRAVITY,wgv,wgv,FLOG,wgv,logManager,0.0,,,,wgv,COMMENT FTOPT LOOP (DB) STOPPED,0,FLOG GRAVITY COMMENT FTOPT LOOP (DB) STOPPED,FLOG GRAVITY COMMENT FTOPT LOOP (DB) STOPPED,1
3,2019-04-02 02:26:36.000,GRAVITY,wgv,wgv,FLOG,wgv,logManager,0.0,,,,wgv,COMMENT LONGITUDINAL PUPIL POSITION LOOP STOPPED,0,FLOG GRAVITY COMMENT LONGITUDINAL PUPIL PO...,FLOG GRAVITY COMMENT LONGITUDINAL PUPIL PO...,1
4,2019-04-02 02:26:36.000,GRAVITY,wgv,wgv,FLOG,wgv,logManager,0.0,,,,wgv,COMMENT FIELD IMAGER LOOP (DB) STOPPED,0,FLOG GRAVITY COMMENT FIELD IMAGER LOOP (DB...,FLOG GRAVITY COMMENT FIELD IMAGER LOOP (DB...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3541701,2019-04-30 22:36:38.422,GRAVITY,wgv,wgv,LOG,wgv,bob_967,187.0,seq,,,,Sending vltdata/tmp/gvmcfg/config/wgv.d...,767,LOG GRAVITY Sending vltdata/tmp/gvmc...,LOG GRAVITY Sending vltdata/tmp/gvmc...,0
3541702,2019-04-30 22:36:38.422,GRAVITY,wgv,wgv,LOG,wgv,bob_967,187.0,seq,,,,Committed revision 321436. (green4),767,LOG GRAVITY Committed revision 321436. (gre...,LOG GRAVITY Committed revision . (green),0
3541703,2019-04-30 22:36:38.426,GRAVITY,wgv,wgv,LOG,wgv,bob_967,187.0,seq,,,,Template Save Instrument Data Bases finished. ...,767,LOG GRAVITY Template Save Instrument Data B...,LOG GRAVITY Template Save Instrument Data B...,0
3541704,2019-04-30 22:36:38.428,GRAVITY,wgv,wgv,LOG,wgv,bob_967,187.0,seq,,,,Finished in 75 seconds at 2019-04-30T22:36:38 ...,767,LOG GRAVITY Finished in 75 seconds at 2019-...,LOG GRAVITY Finished in seconds at T (unde...,0


In [14]:
df_all["trace_id"].value_counts()

,count
trace_id,
74,39304
70,39164
72,33611
78,30253
77,28737
...,...
180,65
340,20
313,15


In [15]:
df_event.describe(include='all')


,@timestamp,event,tokenized
count,3541706,3541706,3541706
unique,NaN,581784,8173
top,NaN,"ERR AT2 seqERR_BAD_QUALITY : ""@wat2tcs:Appl...","ERR AT seqERR_BAD_QUALITY ""@wattcsAppl_dat..."
freq,NaN,229817,283547
mean,2019-04-17 06:48:02.412171264,NaN,NaN
min,2019-04-02 02:26:35.908000,NaN,NaN
25%,2019-04-08 20:54:36.494250240,NaN,NaN
50%,2019-04-19 11:03:28.200499968,NaN,NaN
75%,2019-04-23 00:22:22,NaN,NaN
max,2019-04-30 22:36:38.429000,NaN,NaN


In [16]:
df_combined["TIMEOUT_ERROR"]

,TIMEOUT_ERROR
0,1
1,1
2,1
3,1
4,1
...,...
3541701,0
3541702,0
3541703,0
3541704,0


In [17]:
# Tokenización de los logs (sin números)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_all['tokenized'])

# Entrenar un modelo Naive Bayes
y = df_combined["TIMEOUT_ERROR"]  # columna objetivo


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)

In [19]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predecir y añadir la nueva columna 'NB'
y_test_nb = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_test_nb)


0.811041452316227

In [21]:
df_event["Real"]=y

In [25]:
df_event["NB_1"]=

ValueError: operands could not be broadcast together with shapes (2656279,) (885427,) 

In [23]:
df_event

,@timestamp,event,tokenized,Real,NB
0,2019-04-02 02:26:35.908,LOG GRAVITY GRAVITY_single_acq -- GRAVITY S...,LOG GRAVITY GRAVITY_single_acq GRAVITY Sin...,1,NaN
1,2019-04-02 02:26:35.908,LOG GRAVITY Started at 2019-04-02T02:26:35 ...,LOG GRAVITY Started at T (underlined),1,NaN
2,2019-04-02 02:26:36.000,FLOG GRAVITY COMMENT FTOPT LOOP (DB) STOPPED,FLOG GRAVITY COMMENT FTOPT LOOP (DB) STOPPED,1,NaN
3,2019-04-02 02:26:36.000,FLOG GRAVITY COMMENT LONGITUDINAL PUPIL PO...,FLOG GRAVITY COMMENT LONGITUDINAL PUPIL PO...,1,NaN
4,2019-04-02 02:26:36.000,FLOG GRAVITY COMMENT FIELD IMAGER LOOP (DB...,FLOG GRAVITY COMMENT FIELD IMAGER LOOP (DB...,1,NaN
...,...,...,...,...,...
3541701,2019-04-30 22:36:38.422,LOG GRAVITY Sending vltdata/tmp/gvmc...,LOG GRAVITY Sending vltdata/tmp/gvmc...,0,NaN
3541702,2019-04-30 22:36:38.422,LOG GRAVITY Committed revision 321436. (gre...,LOG GRAVITY Committed revision . (green),0,NaN
3541703,2019-04-30 22:36:38.426,LOG GRAVITY Template Save Instrument Data B...,LOG GRAVITY Template Save Instrument Data B...,0,NaN
3541704,2019-04-30 22:36:38.428,LOG GRAVITY Finished in 75 seconds at 2019-...,LOG GRAVITY Finished in seconds at T (unde...,0,NaN


In [26]:
pip install textblob

In [ ]:
X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 27831641 stored elements and shape (3541706, 4541)>

In [27]:
from textblob import TextBlob


In [ ]:

def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # Retorna el puntaje de sentimiento

# Crear una nueva columna con los puntajes de sentimiento
df_event['SA_1'] = df_event['tokenized'].apply(get_sentiment)

In [ ]:
y_test_nb

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_test_nb)


           NB     SA_1
NB    1.00000  0.14603
SA_1  0.14603  1.00000


In [ ]:
report = df_event[['event','SA_1']].head(10)  # Tomar algunos ejemplos
print(report)

                                               event  NB      SA_1
0   LOG GRAVITY   GRAVITY_single_acq -- GRAVITY S...   1 -0.035714
1   LOG GRAVITY   Started at 2019-04-02T02:26:35 ...   0  0.000000
2    FLOG GRAVITY   COMMENT  FTOPT LOOP (DB) STOPPED   1  0.000000
3   FLOG GRAVITY   COMMENT  LONGITUDINAL PUPIL PO...   1  0.000000
4   FLOG GRAVITY   COMMENT  FIELD IMAGER LOOP (DB...   1  0.000000
5   FLOG GRAVITY   COMMENT  LATERAL PUPIL POSITIO...   1  0.000000
6                  LOG GRAVITY   COU GS FWHM = '0.0'   1  0.000000
7                   LOG GRAVITY   COU AG PMA = '0.0'   1  0.000000
8          LOG GRAVITY   COU AG DELTA = '000000.000'   1  0.000000
9                   LOG GRAVITY   COU AG START = 'F'   1  0.000000
